In [9]:
from pyspark.sql.functions import *
trainDF = spark.read.parquet('/data/user/hive/warehouse/ian/feature/testDF')

trainDF = trainDF.withColumn('m1',substring(trainDF.mac,1,1)).\
      withColumn('m2',substring(trainDF.mac,2,1)).\
      withColumn('m3',substring(trainDF.mac,3,1)).\
      withColumn('m4',substring(trainDF.mac,4,1)).\
      withColumn('m5',substring(trainDF.mac,5,1)).\
      withColumn('m6',substring(trainDF.mac,6,1)).\
      withColumn('m7',substring(trainDF.mac,7,1)).\
      withColumn('m8',substring(trainDF.mac,8,1)).\
      withColumn('m9',substring(trainDF.mac,9,1)).\
      withColumn('m10',substring(trainDF.mac,10,1)).\
      withColumn('m11',substring(trainDF.mac,11,1)).\
      withColumn('m12',substring(trainDF.mac,12,1))

df = trainDF.withColumn('f1',conv(trainDF.m1, 16, 10)).withColumn('f2',conv(trainDF.m2, 16, 10)).\
        withColumn('f3',conv(trainDF.m3, 16, 10)).withColumn('f4',conv(trainDF.m4, 16, 10)).\
        withColumn('f5',conv(trainDF.m5, 16, 10)).withColumn('f6',conv(trainDF.m6, 16, 10)).\
        withColumn('f7',conv(trainDF.m7, 16, 10)).withColumn('f8',conv(trainDF.m8, 16, 10)).\
        withColumn('f9',conv(trainDF.m9, 16, 10)).withColumn('f10',conv(trainDF.m10, 16, 10)).\
        withColumn('f11',conv(trainDF.m11, 16, 10)).withColumn('f12',conv(trainDF.m12, 16, 10))

df = df.select('mac','id',col('f1').cast('float'),col('f2').cast('float')\
               ,col('f3').cast('float'),col('f4').cast('float')\
               ,col('f5').cast('float'),col('f6').cast('float')\
               ,col('f7').cast('float'),col('f8').cast('float')\
               ,col('f9').cast('float'),col('f10').cast('float')\
               ,col('f11').cast('float'),col('f12').cast('float'))

from pyspark.ml.feature import VectorAssembler
vec = VectorAssembler(inputCols=['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12'],outputCol='features')
new_df = vec.transform(df)

from pyspark.ml.feature import StringIndexer
si = StringIndexer(inputCol='id', outputCol='label')
si_model = si.fit(new_df)
df_final = si_model.transform(new_df)

df_final.show(5,truncate=False)

+------------+-----+---+---+---+----+----+----+----+----+---+----+----+---+-------------------------------------------------------+-----+
|mac         |id   |f1 |f2 |f3 |f4  |f5  |f6  |f7  |f8  |f9 |f10 |f11 |f12|features                                               |label|
+------------+-----+---+---+---+----+----+----+----+----+---+----+----+---+-------------------------------------------------------+-----+
|0008221E6CF5|39226|0.0|0.0|0.0|8.0 |2.0 |2.0 |1.0 |14.0|6.0|12.0|15.0|5.0|[0.0,0.0,0.0,8.0,2.0,2.0,1.0,14.0,6.0,12.0,15.0,5.0]   |1.0  |
|000EC6DC7BE2|49814|0.0|0.0|0.0|14.0|12.0|6.0 |13.0|12.0|7.0|11.0|14.0|2.0|[0.0,0.0,0.0,14.0,12.0,6.0,13.0,12.0,7.0,11.0,14.0,2.0]|2.0  |
|001636154AF9|49813|0.0|0.0|1.0|6.0 |3.0 |6.0 |1.0 |5.0 |4.0|10.0|15.0|9.0|[0.0,0.0,1.0,6.0,3.0,6.0,1.0,5.0,4.0,10.0,15.0,9.0]    |5.0  |
|0016785B0DF5|49810|0.0|0.0|1.0|6.0 |7.0 |8.0 |5.0 |11.0|0.0|13.0|15.0|5.0|[0.0,0.0,1.0,6.0,7.0,8.0,5.0,11.0,0.0,13.0,15.0,5.0]   |3.0  |
|0019CBF145D0|49807|0.0|0.0|1.0|9.

In [10]:
df_final.write.mode('overwrite').parquet('hdfs:///data/user/hive/warehouse/ian/feature/testDF_mac_12',compression='gzip')

# Training with non cv

In [1]:
trainDF = spark.read.parquet('/data/user/hive/warehouse/ian/feature/trainDF_mac_12')
testDF = spark.read.parquet('/data/user/hive/warehouse/ian/feature/testDF_mac_12')

In [2]:
trainDF.count(),testDF.count()

(7360274, 3152812)

In [3]:
#from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [4]:
rf = RandomForestClassifier(labelCol='label',featuresCol='features',numTrees=10)

In [5]:
model = rf.fit(trainDF)

In [6]:
predictions = model.transform(testDF)

In [7]:
predictions.dtypes

[('mac', 'string'),
 ('id', 'string'),
 ('f1', 'float'),
 ('f2', 'float'),
 ('f3', 'float'),
 ('f4', 'float'),
 ('f5', 'float'),
 ('f6', 'float'),
 ('f7', 'float'),
 ('f8', 'float'),
 ('f9', 'float'),
 ('f10', 'float'),
 ('f11', 'float'),
 ('f12', 'float'),
 ('features', 'vector'),
 ('label', 'double'),
 ('rawPrediction', 'vector'),
 ('probability', 'vector'),
 ('prediction', 'double')]

In [8]:
predictions.select('label','prediction','mac').show(5,truncate=False)

+-----+----------+------------+
|label|prediction|mac         |
+-----+----------+------------+
|1.0  |2.0       |0008221E6CF5|
|2.0  |2.0       |000EC6DC7BE2|
|5.0  |4.0       |001636154AF9|
|3.0  |0.0       |0016785B0DF5|
|0.0  |1.0       |0019CBF145D0|
+-----+----------+------------+
only showing top 5 rows



In [9]:
evaluator = MulticlassClassificationEvaluator(labelCol='label',predictionCol='prediction',metricName='accuracy')
accuracy = evaluator.evaluate(predictions)

In [10]:
accuracy

0.46540548564265805

# Training with CV & hp tuning Random Forest

In [1]:
trainDF = spark.read.parquet('/data/user/hive/warehouse/ian/feature/trainDF_mac_12')
testDF = spark.read.parquet('/data/user/hive/warehouse/ian/feature/testDF_mac_12')

In [2]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder
import numpy as np

rf = RandomForestClassifier()

paramGrid_rf = ParamGridBuilder()\
       .addGrid(rf.maxDepth,[3,4,5,6,7,8,9,10])\
       .addGrid(rf.numTrees,[10,15,20,25,30])\
       .build()

crossval_rf = CrossValidator(estimator=rf,
                            estimatorParamMaps=paramGrid_rf,
                            evaluator=MulticlassClassificationEvaluator(),
                            numFolds=5)

cvModel_rf = crossval_rf.fit(trainDF)

In [3]:
best_model_rf = cvModel_rf.bestModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
my_eval_rf = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
my_eval_rf.evaluate(best_model_rf.transform(testDF))

0.8072330351444996

In [4]:
best_model_rf.save('hdfs:///data/user/hive/warehouse/ian/model/mac_id_12_rf_cv_tuning_model')

In [5]:
result = best_model_rf.transform(testDF)

In [6]:
result.dtypes

[('mac', 'string'),
 ('id', 'string'),
 ('f1', 'float'),
 ('f2', 'float'),
 ('f3', 'float'),
 ('f4', 'float'),
 ('f5', 'float'),
 ('f6', 'float'),
 ('f7', 'float'),
 ('f8', 'float'),
 ('f9', 'float'),
 ('f10', 'float'),
 ('f11', 'float'),
 ('f12', 'float'),
 ('features', 'vector'),
 ('label', 'double'),
 ('rawPrediction', 'vector'),
 ('probability', 'vector'),
 ('prediction', 'double')]

In [8]:
result.select('mac','id','label','probability','prediction').show(truncate=False)

+------------+-----+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+
|mac         |id   |label|probability                                                                                                                                                                                                                                                                                                                                    |prediction|
+------------+-----+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
result.dtypes

[('mac', 'string'),
 ('id', 'string'),
 ('f1', 'float'),
 ('f2', 'float'),
 ('f3', 'float'),
 ('f4', 'float'),
 ('f5', 'float'),
 ('f6', 'float'),
 ('f7', 'float'),
 ('f8', 'float'),
 ('f9', 'float'),
 ('f10', 'float'),
 ('f11', 'float'),
 ('f12', 'float'),
 ('features', 'vector'),
 ('label', 'double'),
 ('rawPrediction', 'vector'),
 ('probability', 'vector'),
 ('prediction', 'double')]

In [10]:
result.select('id','label','prediction').show(truncate=False)

+-----+-----+----------+
|id   |label|prediction|
+-----+-----+----------+
|39226|1.0  |1.0       |
|49814|2.0  |1.0       |
|49813|5.0  |1.0       |
|49810|3.0  |1.0       |
|49807|0.0  |1.0       |
|49808|6.0  |10.0      |
|10679|15.0 |1.0       |
|49807|0.0  |0.0       |
|49807|0.0  |0.0       |
|49807|0.0  |0.0       |
|49807|0.0  |0.0       |
|10680|10.0 |10.0      |
|10680|10.0 |10.0      |
|10680|10.0 |10.0      |
|10680|10.0 |10.0      |
|39147|7.0  |0.0       |
|39226|1.0  |0.0       |
|39000|8.0  |0.0       |
|49807|0.0  |0.0       |
|49807|0.0  |0.0       |
+-----+-----+----------+
only showing top 20 rows



In [11]:
mapping_table = result.select('id','label').distinct()

In [10]:
#mapping_table.write.mode('overwrite').parquet('hdfs:///data/user/hive/warehouse/ian/feature/mapping_table',compression='gzip')

In [12]:
mapping_table = mapping_table.withColumnRenamed('id','predict_id')

In [13]:
result = result.select('mac','prediction')
final_result = result.join(mapping_table,result.prediction == mapping_table.label)

In [14]:
final_result = final_result.select('mac','predict_id').distinct()

In [15]:
final_result.show(truncate=False)

+------------+----------+
|mac         |predict_id|
+------------+----------+
|6C4D7347102D|39147     |
|70EF008B3FB1|39147     |
|70EF00B1ACC9|39147     |
|70EF00CFC4E4|39147     |
|94F6D67C1BF1|39147     |
|B019C6195BC1|39147     |
|C0A53E63D790|39147     |
|D461DA06B4E1|39147     |
|F895EA6A5E84|39147     |
|347C2553D215|39147     |
|586B14515ABE|39147     |
|94F6D639DC93|39147     |
|C0A53E6AB23B|39147     |
|CC2DB7BBBE22|39147     |
|CC2DB7E61415|39147     |
|3C2EF9520D37|39147     |
|40261924FFAB|39147     |
|94F6D62AB011|39147     |
|C0A53E93B510|39147     |
|CC2DB7467566|39147     |
+------------+----------+
only showing top 20 rows



In [49]:
testDF.filter(testDF.mac == '5433CB525C01').show()

+------------+-----+---+---+---+---+----+----+---+---+---+----+---+---+--------------------+-----+
|         mac|   id| f1| f2| f3| f4|  f5|  f6| f7| f8| f9| f10|f11|f12|            features|label|
+------------+-----+---+---+---+---+----+----+---+---+---+----+---+---+--------------------+-----+
|5433CB525C01|39147|5.0|4.0|3.0|3.0|12.0|11.0|5.0|2.0|5.0|12.0|0.0|1.0|[5.0,4.0,3.0,3.0,...|  7.0|
+------------+-----+---+---+---+---+----+----+---+---+---+----+---+---+--------------------+-----+



# crontab mac_id prediction

In [37]:
from pyspark.ml.classification import RandomForestClassificationModel
model = RandomForestClassificationModel.load('/data/user/hive/warehouse/ian/model/mac_id_12_rf_cv_tuning_model')
testDF = spark.read.parquet('/data/user/hive/warehouse/ian/feature/testDF_mac_12') #change PATH
result = model.transform(testDF)
mapping_table = spark.read.parquet('/data/user/hive/warehouse/ian/feature/mapping_table').withColumnRenamed('id','predict_id')
result = result.select('mac','prediction')
final_result = result.join(mapping_table,result.prediction == mapping_table.label)
final_result = final_result.select('mac','predict_id').distinct()
final_result.sample(0.001).show(truncate=False)

+------------+----------+
|mac         |predict_id|
+------------+----------+
|9CE33F6AF89B|39226     |
|84A134385E87|49810     |
|5433CB525C01|39226     |
|70F087774375|49814     |
|68DBCA53E6A9|49810     |
|84A134093602|49810     |
|C83C85E5A5F1|39226     |
|404D7FE74BEE|49813     |
|E0ACCB434FF3|49880     |
|7C04D056B17E|49814     |
|24F67750A0A7|49814     |
|38C98692F03A|49880     |
|94F6D6723318|39147     |
|90B0EDB39F67|49810     |
|68AB1E19EAB0|49880     |
|9CE33F951EEA|39226     |
|7C50495799BF|49814     |
|DCA4CAD4D097|49814     |
|70EF00427A61|39147     |
|DC0C5C8D8A23|49814     |
+------------+----------+
only showing top 20 rows



In [38]:
final_result.groupBy('predict_id').count().show(truncate=False)

+----------+------+
|predict_id|count |
+----------+------+
|49810     |332712|
|39890     |65343 |
|49809     |320635|
|49866     |60898 |
|49880     |84506 |
|39147     |225601|
|49807     |415662|
|39000     |24245 |
|49814     |402567|
|49813     |209372|
|39226     |425120|
|10679     |25368 |
|10950     |139825|
|10680     |126929|
|49806     |74259 |
|49808     |219770|
+----------+------+



# OnevsRest(LR)

In [2]:
from  pyspark.sql.functions import *
trainDF = spark.read.parquet('/data/user/hive/warehouse/ian/feature/trainDF')
testDF = spark.read.parquet('/data/user/hive/warehouse/ian/feature/testDF')
from pyspark.ml.classification import OneVsRest,LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder
import numpy as np

svc = LinearSVC()

ovr = OneVsRest(classifier=svc)

model = ovr.fit(trainDF)

In [8]:
#model.save('hdfs:///data/user/hive/warehouse/ian/model/mac_id_ovr_svc_model')

In [8]:
from pyspark.sql.functions import *
from pyspark.ml.classification import OneVsRestModel
model = OneVsRestModel.load('hdfs:///data/user/hive/warehouse/ian/model/mac_id_ovr_svc_model')
testDF = spark.read.parquet('/data/user/hive/warehouse/ian/feature/testDF')
result = model.transform(testDF.sample(0.001))

In [9]:
df = result.withColumn('compare',when(result.label == result.prediction,1).otherwise(0)).select('mac','cn_name','label','prediction','compare')

In [10]:
df.groupBy('compare').count().show()

+-------+-----+
|compare|count|
+-------+-----+
|      1|  343|
|      0| 2836|
+-------+-----+



# OnevsRest(NB)

In [1]:
from  pyspark.sql.functions import *
trainDF = spark.read.parquet('/data/user/hive/warehouse/ian/feature/trainDF')
testDF = spark.read.parquet('/data/user/hive/warehouse/ian/feature/testDF')

from pyspark.ml.classification import OneVsRest,NaiveBayes

nb = NaiveBayes()

ovr = OneVsRest(classifier=nb)

model = ovr.fit(trainDF)

In [2]:
#model.save('hdfs:///data/user/hive/warehouse/ian/model/mac_id_nb_model')

In [3]:
from pyspark.ml.classification import OneVsRestModel
from pyspark.sql.functions import *
model = OneVsRestModel.load('/data/user/hive/warehouse/ian/model/mac_id_nb_model')
testDF = spark.read.parquet('/data/user/hive/warehouse/ian/feature/testDF')
result = model.transform(testDF.sample(0.001))
df = result.withColumn('compare',when(result.label == result.prediction,1).otherwise(0)).select('mac','cn_name','label','prediction','compare')
df.groupBy('compare').count().show()

+-------+-----+
|compare|count|
+-------+-----+
|      1|  402|
|      0| 2812|
+-------+-----+

